In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd ./drive/MyDrive/

/content/drive/MyDrive/Rub


In [ ]:
!pip install ftfy
!pip install emoji

In [ ]:
import json
import os, sys
import html
import nltk
import random
import re
import pandas as pd
import numpy as np
from collections import Counter
from nltk.tokenize import TweetTokenizer
from ftfy import fix_text
from sutime import SUTime

# Text 

In [ ]:
train_dict = {}
test_dict = {}

In [ ]:
with open('/content/drive/MyDrive/data/training.csv', encoding="utf8") as f:
  train_data = f.readlines()[1:]
with open('/content/drive/MyDrive/data/test.csv', encoding="utf8") as f:
  test_data = f.readlines()[1:]

In [ ]:
def cleanup_text(text):

  # 1.Replace common web abbreviations with their original meaning
  f = open('/content/drive/MyDrive/data/slang.txt', "r")
  chat_words_str = f.read()
  chat_words_map_dict = {}   
  chat_words_list = []     

  for line in chat_words_str.split("\n"):
      if line != "":
          cw = line.split("=")[0]
          cw_expanded = line.split("=")[1]
          chat_words_list.append(cw)
          chat_words_map_dict[cw] = cw_expanded
  chat_words_list = set(chat_words_list)

  def chat_words_conversion(text):
      new_text = []
      for w in text.split():
          if w.upper() in chat_words_list:
              new_text.append(chat_words_map_dict[w.upper()])
          else:
              new_text.append(w)
      return " ".join(new_text)

  # 2.delete Twitter Usernames, website, Multiple spaces, watermarks

  def remove_strings(text):
    meaningless_strings = ["Twitter for iPhone", "VERY DEMOTIVATIONAL.com", "made with mematic","Meme Center","MemeCenter"]
    
    # Twitter Usernames
    text = re.sub("@(\w){1,15}", "", text) 

    rep = [re.escape(x) for x in meaningless_strings]
            
    text = re.sub("([A-Za-z0-9]\.|[A-Za-z0-9][A-Za-z0-9-]{0,61}[A-Za-z0-9]\.){1,3}(com|net|org|co\.uk|co|ru|eu)", "", text, flags=re.IGNORECASE)    

    pattern = re.compile("|".join(rep), flags=re.IGNORECASE)
    text = pattern.sub("", text)
    text = re.sub("\s\s+"," ", text) # Multiple spaces
    return text

  # 3.delete timestamps
  def remove_timestamps(sutime, text):
    jar_files = os.path.join(os.path.dirname('/usr/local/lib/python3.6/dist-packages/sutime/jars'), 'jars')
    sutime = SUTime(jars=jar_files, mark_time_ranges=True)
    date_seq = sutime.parse(text)
    res = text
    for seq in date_seq:
        if seq['type'] in ['DATE', 'TIME', 'DURATION']:
          res = text[:seq['start']] + " "*(seq['end'] - seq['start']) + text[seq['end']:]
    if len(date_seq):
        res = " ".join(res.split())
    return res

  # 4.delete non-English
  def nospecial(text):
    import re
    text = re.sub("[^a-zA-Z0-9 .,?!\']+", "",text)
    return text
  
  text = chat_words_conversion(text)
  text = remove_timestamps(sutime, text)
  text = remove_strings(text)
  text = nospecial(text)

  return text.lower()

In [ ]:
for i in train_data:
  split = i.split('\t')
  file_id = split[0]
  misogynous = split[1]
  texttrans = split[6].strip('\n')
  texttrans = fix_text(texttrans)
  texttrans = cleanup_text(texttrans)

  train_dict.update({file_id: {}})
  train_dict[file_id].update({'misogynous': misogynous})
  train_dict[file_id].update({'text': texttrans})

In [ ]:
len(train_dict)

In [ ]:
for i in test_data:
  split = i.split('\t')
  file_id = split[0]
  texttrans = split[1].strip('\n')
  texttrans = fix_text(texttrans)
  texttrans = cleanup_text(texttrans)

  test_dict.update({file_id: {}})
  test_dict[file_id].update({'misogynous': misogynous})
  test_dict[file_id].update({'text': texttrans})

In [ ]:
len(test_dict)

In [ ]:
for key,value in train_dict.items():
  id = key[:-4]
  img = 'img/' + key
  value['id'] = id
  value['img'] = img

In [ ]:
train = []
for value in train.values():
  train.append(value)

In [ ]:
import random
from random import shuffle
shuffle(train)

[{'text': "smoke detector that shuts off when you yell i'm just cooking invest", 'label': '0', 'id': '2000', 'img': 'img/2000.jpg'}, {'text': 'winner bronze medal trophy wife made of bronze very ', 'label': '1', 'id': '1500', 'img': 'img/1500.jpg'}, {'text': 'foot long inb4 blowjob jokes 3.00 rn whoooops too late.', 'label': '1', 'id': '6848', 'img': 'img/6848.jpg'}, {'text': 'tv brown amirah stay away from my girl! their girl', 'label': '1', 'id': '9479', 'img': 'img/9479.jpg'}, {'text': 'if you are depressed and single a relationship will not change that ', 'label': '0', 'id': '6369', 'img': 'img/6369.jpg'}, {'text': "biden president americansforliberty76 i'm joe biden, and i forgot this message", 'label': '0', 'id': '5478', 'img': 'img/5478.jpg'}, {'text': 'free dishwasher missing parts marepererador', 'label': '1', 'id': '8496', 'img': 'img/8496.jpg'}, {'text': 'found at yeah i believe god is a woman. only a woman could be so emotionally unstable. ', 'label': '1', 'id': '175', 'img

In [ ]:
import jsonlines
def parse(self, response):
    jsonresponse=json.loads(response.text)
    with jsonlines.open('output.jsonl',mode='a') as writer:
        writer.write(jsonresponse)

In [ ]:
import json

def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

In [ ]:
dev = train[:1000]
final_train = train[1000:]

In [ ]:
dump_jsonl(final_train, '/content/drive/MyDrive/data/train.jsonl', append=True)
dump_jsonl(dev, '/content/drive/MyDrive/data/dev.jsonl', append=True)
dump_jsonl(test, '/content/drive/MyDrive/data/test.jsonl', append=True)